In [1]:
import requests
import pandas as pd
import numpy as np
import json
import time

In [2]:
from config import api_key

target_url = f'http://api.sportradar.us/nba/trial/v8/en/seasons/2022/REG/teams/583eca2f-fb46-11e1-82cb-f4ce4684ea4c/statistics.json?api_key={api_key}'

timberwolves_data = requests.get(target_url).json()
print(json.dumps(timberwolves_data, indent=4))


{
    "id": "583eca2f-fb46-11e1-82cb-f4ce4684ea4c",
    "name": "Timberwolves",
    "market": "Minnesota",
    "sr_id": "sr:team:3426",
    "reference": "1610612750",
    "season": {
        "id": "5027b6ac-731c-4622-8d69-d863ae7c626b",
        "year": 2022,
        "type": "REG"
    },
    "own_record": {
        "total": {
            "games_played": 82,
            "minutes": 19830.0,
            "field_goals_made": 3515,
            "field_goals_att": 7167,
            "field_goals_pct": 0.49,
            "two_points_made": 2518,
            "two_points_att": 4436,
            "two_points_pct": 0.568,
            "three_points_made": 997,
            "three_points_att": 2731,
            "three_points_pct": 0.365,
            "blocked_att": 336,
            "free_throws_made": 1467,
            "free_throws_att": 1942,
            "free_throws_pct": 0.755,
            "offensive_rebounds": 749,
            "defensive_rebounds": 2686,
            "rebounds": 3435,
            "assis

In [4]:
team_url = f'http://api.sportradar.us/nba/trial/v8/en/seasons/2022/REG/standings.json?api_key={api_key}'

team_data = requests.get(team_url).json()
def extract_team_data(data):
    teams = []
    if 'teams' in data:
        for team in data['teams']:
            team_id = team['id']
            team_name = team['name']
            teams.append({'id': team_id, 'name': team_name})
    if 'divisions' in data:
        for division in data['divisions']:
            teams += extract_team_data(division)
    if 'conferences' in data:
        for conference in data['conferences']:
            teams += extract_team_data(conference)
    return teams

# Extract team id and team name from the team_data dictionary
teams = extract_team_data(team_data)
teams_info=pd.DataFrame(teams)
teams_id= teams_info["id"]


In [14]:
# Initialize
team_name=[]
team_field_goals=[]
team_id=[]
team_data={}
team_offensive_rebounds=[]
team_turnovers=[]
team_free_throws_att=[]
team_possession=[]
i=0
# Loop through each team ID and retrieve their statistics from the API
for i in range(len(teams_id)):
    team_url = f'http://api.sportradar.us/nba/trial/v8/en/seasons/2022/REG/teams/{teams_id[i]}/statistics.json?api_key={api_key}'
    team_data = requests.get(team_url).json()
    # Extract various statistics for the each team
    field_goals_att=team_data['own_record']['total']['field_goals_att']
    offensive_rebounds=team_data['own_record']['total']['offensive_rebounds']
    turnovers= team_data['own_record']['total']['turnovers']
    free_throws_att= team_data['own_record']['total']['free_throws_att']

    #calculate Possession for each team
    possession = 0.96*(field_goals_att + turnovers + (0.44*free_throws_att) -offensive_rebounds)
    team_totals_dict = team_data['own_record']['total']
    opp_vs_team_totals_dict = team_data['opponents']['total']

    team_stat_keys = ['games_played', 'minutes', 'field_goals_made', 'field_goals_att',
                      'three_points_made', 'three_points_att', 'three_points_pct',
                      'free_throws_made', 'free_throws_att', 'offensive_rebounds',
                      'defensive_rebounds', 'rebounds', 'assists', 'turnovers',
                      'steals', 'blocks', 'points']
    team_stats = {k: v for k, v in team_totals_dict.items() if k in team_stat_keys}
    opp_vs_team_stats = {k: v for k, v in opp_vs_team_totals_dict.items() if k in team_stat_keys}

    #team stats calculations
    poss = 0.96 * ((team_stats['field_goals_att']) - (team_stats['offensive_rebounds']) + (team_stats['turnovers']) + 0.44 * (team_stats['free_throws_att']))
    team_o_rtg = 100 * (team_stats['points']) / (poss)
    team_d_rtg = 100 * (opp_vs_team_stats['points']) / (poss)
    team_net_rtg = (team_o_rtg) - (team_d_rtg)

    #Append the extracted statistics to teams' lists
    team_name.append(team_data['name'])
    team_id.append(team_data['id']) 
    team_field_goals.append(field_goals_att)
    team_offensive_rebounds.append(offensive_rebounds)
    team_turnovers.append(turnovers)
    team_free_throws_att.append(free_throws_att)
    team_possession.append(possession)

    # Pause execution for 0.5 second to avoid overloading the API
    time.sleep(1)  

# Create Teams DataFrame
teams_df=pd.DataFrame({
    'Team Name' : team_name,
    #'Team ID': team_id,
    'Team Possession': team_possession,
    'team_o_rtg': team_o_rtg,
    'team_d_rtg': team_d_rtg,
    'team_net_rtg': team_net_rtg ,
        
    'Team Field Goals Attempts':team_field_goals,
    'Team Offensive Rebounds': team_offensive_rebounds,
    'Team Turnovers': team_turnovers,
    'Team Free throws Attemps': team_free_throws_att

})
teams_df

,Team Name,Team Possession,team_o_rtg,team_d_rtg,team_net_rtg,Team Field Goals Attempts,Team Offensive Rebounds,Team Turnovers,Team Free throws Attemps
0,Grizzlies,8143.4496,117.388687,115.85378,1.534907,7551,988,1060,1954
1,Pelicans,8009.2800,117.388687,115.85378,1.534907,7180,866,1149,2000
2,Mavericks,7820.2368,117.388687,115.85378,1.534907,6909,625,957,2057
3,Rockets,8033.5872,117.388687,115.85378,1.534907,7287,1100,1267,2078
4,Spurs,8257.1136,117.388687,115.85378,1.534907,7593,965,1208,1739
5,Kings,8092.9920,117.388687,115.85378,1.534907,7232,782,1076,2055
6,Suns,7925.4144,117.388687,115.85378,1.534907,7388,971,1055,1781
7,Clippers,7900.1088,117.388687,115.85378,1.534907,7059,803,1110,1962
8,Warriors,8202.4320,117.388687,115.85378,1.534907,7393,864,1287,1655
9,Lakers,8202.3168,117.388687,115.85378,1.534907,7298,818,1104,2182


In [15]:
standing_url = f'http://api.sportradar.us/nba/trial/v8/en/seasons/2022/REG/standings.json?api_key=aur7d7bebpgz8m78sxs5z5zh'
standing_data = requests.get(standing_url).json()
aa= standing_data["conferences"]
Teams_name=[]
standing_rank=[]
conferences2=[]
win=[]
n=i=j=0

for n in range(len(aa)):
    zz= standing_data["conferences"][n]['divisions']
    
    
    for i in range(len(zz)):
        bb=standing_data["conferences"][n]['divisions'][i]['teams']
        

        for j in range(len(bb)):
            Teams_name.append(standing_data["conferences"][n]['divisions'][i]['teams'][j]['name'])
            standing_rank.append(standing_data["conferences"][n]['divisions'][i]['teams'][j]['calc_rank']['conf_rank'])
            win.append(standing_data["conferences"][n]['divisions'][i]['teams'][j]['wins'])
            conferences2.append(standing_data["conferences"][n]['alias'])
    #b=pd.DataFrame[b]
    standing_df=pd.DataFrame({"Team Name": Teams_name,
                'Wins': win,
                'Conference Ranking': standing_rank,
                'Conferences':conferences2

   })
standing_df

,Team Name,Wins,Conference Ranking,Conferences
0,Grizzlies,51,2,WESTERN
1,Pelicans,42,10,WESTERN
2,Mavericks,38,11,WESTERN
3,Rockets,22,14,WESTERN
4,Spurs,22,15,WESTERN
5,Kings,48,3,WESTERN
6,Suns,45,4,WESTERN
7,Clippers,44,5,WESTERN
8,Warriors,44,6,WESTERN
9,Lakers,43,7,WESTERN


In [16]:
last_team_df=pd.merge(standing_df,teams_df,on="Team Name")
last_team_df=last_team_df.sort_values('Conference Ranking')
last_team_df

,Team Name,Wins,Conference Ranking,Conferences,Team Possession,team_o_rtg,team_d_rtg,team_net_rtg,Team Field Goals Attempts,Team Offensive Rebounds,Team Turnovers,Team Free throws Attemps
15,Bucks,58,1,EASTERN,8104.5888,117.388687,115.85378,1.534907,7411,914,1137,1837
10,Nuggets,53,1,WESTERN,7872.9216,117.388687,115.85378,1.534907,7088,828,1134,1834
0,Grizzlies,51,2,WESTERN,8143.4496,117.388687,115.85378,1.534907,7551,988,1060,1954
25,Celtics,57,2,EASTERN,7971.2256,117.388687,115.85378,1.534907,7278,796,1043,1769
26,76ers,54,3,EASTERN,7788.0192,117.388687,115.85378,1.534907,6870,715,1052,2058
5,Kings,48,3,WESTERN,8092.9920,117.388687,115.85378,1.534907,7232,782,1076,2055
6,Suns,45,4,WESTERN,7925.4144,117.388687,115.85378,1.534907,7388,971,1055,1781
16,Cavaliers,51,4,EASTERN,7699.5456,117.388687,115.85378,1.534907,6984,795,1020,1844
27,Knicks,47,5,EASTERN,7875.1488,117.388687,115.85378,1.534907,7328,1031,988,2087
7,Clippers,44,5,WESTERN,7900.1088,117.388687,115.85378,1.534907,7059,803,1110,1962
